<a href="https://colab.research.google.com/github/primalbioinformatics/drug-informatics/blob/main/chemb_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install chembl_webresource_client
! pip install rdkit
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from rdkit.Chem.MolStandardize import rdMolStandardize
from chembl_webresource_client.new_client import new_client
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.5 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=6efd39948612009a94b7a2998011623eda13fbe6dfce5bef5dad649480321fa3
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1


In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id="CHEMBL262").filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)
df.head(3)
df.standard_type.unique()
df = df[df.standard_value.notna()]
df['standard_value'] = df['standard_value'].astype(float)
df['bioactivity_class'] = df['standard_value'].map(lambda x : 'active' if x <= 1000 else ('inactive' if x >=10000 else 'intermediate'))
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value','bioactivity_class']
df = df[selection]
df.head(5)
df.canonical_smiles.replace('nan',np.nan, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import traceback
from mordred import Calculator, descriptors

def lipinski(smiles, verbose=False):
    moldata = []
    problematic_indices = []
    for i, elem in enumerate(smiles):
        try:
            mol = Chem.MolFromSmiles(elem)
            mol = rdMolStandardize.Cleanup(mol)
            moldata.append(mol)
        except:
            problematic_indices.append(i)
            pass

    baseData = np.arange(1, 1)
    for mol in moldata:
        desc = Calculator(descriptors)(mol)  # Setting ignore_3D=True
        row = np.array(list(desc.values()))

        if baseData.size == 0:
            baseData = row
        else:
            baseData = np.vstack([baseData, row])

    # Get the names of the descriptors
    descriptor_names = list(desc.keys())

    desc = pd.DataFrame(data=baseData, columns=descriptor_names)

    # Remove problematic rows
    desc.drop(problematic_indices, inplace=True)

    return desc

df_lipinski = lipinski(df.canonical_smiles)


Streaming output truncated to the last 5000 lines.
[14:27:22] Initializing Normalizer
[14:27:22] Running Normalizer
[14:27:22] Initializing MetalDisconnector
[14:27:22] Running MetalDisconnector
[14:27:22] Initializing Normalizer
[14:27:22] Running Normalizer
[14:27:22] Initializing MetalDisconnector
[14:27:22] Running MetalDisconnector
[14:27:22] Initializing Normalizer
[14:27:22] Running Normalizer
[14:27:22] Initializing MetalDisconnector
[14:27:22] Running MetalDisconnector
[14:27:22] Initializing Normalizer
[14:27:22] Running Normalizer
[14:27:22] Initializing MetalDisconnector
[14:27:22] Running MetalDisconnector
[14:27:22] Initializing Normalizer
[14:27:22] Running Normalizer
[14:27:22] Initializing MetalDisconnector
[14:27:22] Running MetalDisconnector
[14:27:22] Initializing Normalizer
[14:27:22] Running Normalizer
[14:27:22] Initializing MetalDisconnector
[14:27:22] Running MetalDisconnector
[14:27:22] Initializing Normalizer
[14:27:22] Running Normalizer
[14:27:22] Initializ

In [ ]:
df_lipinski.head(10).style.background_gradient()

In [ ]:
df.shape

(3780, 5)

In [ ]:
df = pd.concat([df,df_lipinski], axis=1)

In [ ]:
df.columns

Index([             'index', 'molecule_chembl_id',   'canonical_smiles',
           'standard_value',  'bioactivity_class',                  ABC,
                      ABCGG,                nAcid,                nBase,
                    SpAbs_A,
       ...
                      SRW10,               TSRW10,                   MW,
                        AMW,                WPath,                 WPol,
                    Zagreb1,              Zagreb2,             mZagreb1,
                   mZagreb2],
      dtype='object', length=1618)

In [ ]:
def pIC50(i):
    molar = i*(10**-9) # Converts nM to M
    pIC50 = -np.log10(molar)
    return pIC50

In [ ]:

df.standard_value.describe()

count    3.780000e+03
mean     4.213967e+04
std      1.153058e+06
min      1.300000e-02
25%      5.775000e+01
50%      6.900000e+02
75%      9.412500e+03
max      5.000000e+07
Name: standard_value, dtype: float64

In [ ]:
df['norm_value'] = df['standard_value'].map(lambda x: 100000000 if x> 100000000 else x)
df['pIC50'] = df['norm_value'].map(pIC50)

In [ ]:
df = df[df['bioactivity_class'] != 'intermediate']
df['bioactivity_class'].value_counts()

active      2107
inactive     931
Name: bioactivity_class, dtype: int64

In [ ]:
num_columns = df.shape[1]
print("Number of columns in df:", num_columns)


Number of columns in df: 1620


In [ ]:
# Assuming df_lipinski is the DataFrame you want to save
df.to_csv('mordred_desc_chembl262.csv', index=False)


In [ ]:
df.columns


Index([             'index', 'molecule_chembl_id',   'canonical_smiles',
           'standard_value',  'bioactivity_class',                  ABC,
                      ABCGG,                nAcid,                nBase,
                    SpAbs_A,
       ...
                         MW,                  AMW,                WPath,
                       WPol,              Zagreb1,              Zagreb2,
                   mZagreb1,             mZagreb2,         'norm_value',
                    'pIC50'],
      dtype='object', length=1620)

In [ ]:
import pandas as pd

# Load the data
df1 = pd.read_csv('/content/mordred_desc_chembl262.csv', index_col=0)
df2 = pd.read_csv('/content/rdkit.csv', index_col=0)

# Initialize a DataFrame to store columns with identical values
matching_columns = pd.DataFrame()

# Keep track of columns already added to avoid duplicates
added_columns = set()

# Compare each column in df1 with each column in df2
for col1 in df1.columns:
    for col2 in df2.columns:
        if col1 not in added_columns and col2 not in added_columns:
            # Check if columns have the same values (ignoring the order and duplicates)
            if sorted(df1[col1].dropna().unique()) == sorted(df2[col2].dropna().unique()):
                # Add the column from df1 to the matching_columns DataFrame
                matching_columns[col1] = df1[col1]
                # Mark these columns as added to avoid duplication
                added_columns.add(col1)
                added_columns.add(col2)
                break  # Break the inner loop if a match is found

# Save the matching columns to a new CSV file
matching_columns.to_csv('chembl262_all_descriptors.csv')


<ipython-input-16-c40b346d2856>:4: DtypeWarning: Columns (0,1184,1185,1186,1187,1188,1393) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('/content/mordred_rdkit (1).csv')
<ipython-input-16-c40b346d2856>:5: DtypeWarning: Columns (140,141,142,143,144,145,149,150,151,152,153,154,167,168,169,170,171,172,176,177,178,179,180,181,185,186,187,188,189,190,194,195,196,197,198,199,203,204,205,206,207,208,212,213,214,215,216,217,221,222,223,224,225,226,230,231,232,233,234,235,347,348,349,350,351,352,356,357,358,359,360,361,365,366,367,368,369,370,383,384,385,386,387,388,392,393,394,395,396,397,401,402,403,404,405,406,410,411,412,413,414,415,419,420,421,422,423,424,428,429,430,431,432,433,437,438,439,440,441,442,446,447,448,449,450,451,454,455,456,457,458,459,462,463,464,465,466,467,470,471,472,473,474,475,486,487,488,489,490,491,494,495,496,497,498,499,502,503,504,505,506,507,510,511,512,513,514,515,518,519,520,521,522,523,526,527,528,529,530,531,534

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from tqdm import tqdm
from sklearn.metrics import r2_score

x_df = pd.read_csv('calculated_descriptors.csv', index_col='ID')

pairs = list(combinations(x_df.columns, 2))

remove = []

for pair in tqdm(pairs):

    a0 = x_df[pair[0]].values
    a1 = x_df[pair[1]].values

    if r2_score(a1, a0) > 0.95:
        remove.append(pair[0])

remove = list(dict.fromkeys(remove))

x_df.drop(columns=remove, inplace=True)

x_df.to_csv('unique_descriptors_chembl262_mordred.csv')

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

np.seterr(divide='ignore')


class RReliefF:

    def __init__(self, k=10, sigma=30):

        self.k = k
        self.sigma = sigma

        self.weights = None
        self.w_track = None

        distance_list = [np.exp(-((n + 1) / sigma) ** 2) for n in range(k)]
        self._distance = distance_list / np.sum(distance_list)

    def fit(self, x, y):

        m = x.shape[0]

        self.weights = np.zeros([x.shape[1],1])
        self.w_track = np.zeros([m, x.shape[1]])

        n_dc = 0
        n_da = np.zeros([x.shape[1],1])
        n_dcandda= np.zeros([x.shape[1],1])
        y_range = np.max(y) - np.min(y)

        for i in tqdm(range(m)):

            b = x[i, :]
            difference = (x - b)**2
            sum_difference = np.sum(difference, axis = 1)
            neighbour_index = np.argsort(sum_difference)
            neighbours = x[neighbour_index][1:]
            knn = neighbours[:self.k]

            x_knn, neighbour_index = knn, neighbour_index[1:]
            y_knn = y[neighbour_index]

            x_i = x[i, :]
            y_i = y[i]

            for j in range(self.k):

                n_dc += (np.abs(y_i-y_knn[j])/y_range) * self._distance[j]

                for a in range(x.shape[1]):

                    diff = np.abs(x_i[a] - x_knn[j][a]) / np.max(x[:, a]) - np.min(x[:, a])

                    n_da[a] = n_da[a] + self._distance[j] * diff

                    n_dcandda[a] = n_dcandda[a] + (np.abs(y_i-y_knn[j])/y_range) * self._distance[j] * diff

            for a in range(x.shape[1]):
                self.w_track[i, a] = n_dcandda[a] / n_dc - ((n_da[a] - n_dcandda[a]) / (m - n_dc))

        for a in range(x.shape[1]):
            self.weights[a] = n_dcandda[a]/n_dc - ((n_da[a]-n_dcandda[a])/(m-n_dc))


x = StandardScaler().fit_transform(pd.read_csv('/content/unique_descriptors_chembl262_mordred.csv', index_col='molecule_chembl_id').values)
y = pd.read_csv('/content/pic50_chemblid_target - Sheet1.csv', index_col='molecule_chembl_id').values
y.shape
method = RReliefF()
method.fit(x, y)

data = dict(Descriptor=list(pd.read_csv('/content/unique_descriptors_chembl262_mordred.csv', index_col='molecule_chembl_id').columns),
            Values=list(np.ravel(method.weights)))

df = pd.DataFrame.from_dict(data)
df.to_csv('rrelieff.csv', index=False)

  0%|          | 0/2307 [00:00<?, ?it/s]<ipython-input-24-ba63d5e91083>:56: RuntimeWarning: invalid value encountered in scalar divide
  diff = np.abs(x_i[a] - x_knn[j][a]) / np.max(x[:, a]) - np.min(x[:, a])
<ipython-input-24-ba63d5e91083>:63: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.w_track[i, a] = n_dcandda[a] / n_dc - ((n_da[a] - n_dcandda[a]) / (m - n_dc))
100%|██████████| 2307/2307 [14:54<00:00,  2.58it/s]
